# [Métodos Heurísticos para Optimización] Solución Parcial 2020-2

##### Estudiante:
- Pablo A. Saldarriaga Aristizábal

En este notebook se realiza la solución del parcial de métodos heurísticos para optimización.

In [1]:
### paquetes para correr el notebook
import math
import numpy as np
from random import choice

# Punto 1 (30%) 

##### Considere el algoritmo descrito en el artículo asignado: “A bi-population EDA for solving the no-idle permutation flow-shop scheduling problem with the total tardiness criterion” (enviado adjunto con el examen). Clasifique sus diferentes componentes de acuerdo a su categoría (diversificación o intensificación).

En el paper proponen un nuevo algoritmo como la modificación deuno ya existente (EDA), se menciona que el algoritmo original presenta una fortaleza en términos de "exploración" (es decir, diversificación), ya que en cada iteración considera un método probabilístico para realizar un nuevo muestreo a la población y así considerar diferentes zonas del espacio de soluciones. Por lo tanto, el algoritmo que proponen (BEDA) considera esta aproximación, además de realizar en cada iteración una búsqueda local en el mejor individuo de la población (intensificación), además de agregar otro factor aleatorio al momento de realizar la actualización de la población, así para actualizar la población en cada iteración, lo que hace es tomar con cierta probabilidad individuos de la población original tal como lo hace el método EDA (hace una exploración global), y por otro lado hace una selección de individuos teniendo como base la mejor solución explorada (por lo que en este caso, este proceso corresponde a intensificación, ya que busca continuar explorando un buen espacio de soluciones). Teniendo esto en mente, a continuación se realiza la respectiva clasificación de las componentes del algorítmo de acuerdo a su categoría:

<img src="images/BEDA_clasificacion.png">

# Punto 2 (10%)

##### (10%) Indique a cuál metaheurístico corresponde el siguiente pseudo-código.

<img src="images/algoritmo.png">

El algoritmo correspondiente al pseudo-código enviado corresponde al Recocido Simulado (o Simulatead Annealing)

# Punto 3 (30%)

##### De acuerdo con el método del numeral anterior, indique cual sería la modificación necesaria si el método aplicara su criterio de aceptación sobre la mejor entre m soluciones generadas.

<img src="images/Modificacion_algoritmo.png">

# Punto 4 (30%)

##### Dado el siguiente problema de optimización:

$$
Min Z = e^{x_{1}x_{2}x_{3}x_{4}x_{5}}
$$

S.A

$$ x_{1}^{2}+x_{2}^{2}+x_{3}^{2}+x_{4}^{2}+x_{5}^{2}-10=0 $$
$$x_{2}x{3}-5x_{4}x_{5} = 0$$
$$ x_{1}^{3}+x_{2}^{3}+1=0$$
$$ l_{i}\leq x_{i} \leq u_{i} $$
$$ l_{i} = -u_{1} $$
$$ u_{i}= [2.3, 2.3, 3.2, 3.2, 3.2]$$

##### Y la siguiente relajación del problema:

$$
Min Z^{R} = e^{x_{1}x_{2}x_{3}x_{4}x_{5}} + e^{g_{1}(X)+g_{3}(X)+g_{3}(X)+h_{1}(X)+h_{2}(X)+h_{3}(X)+h_{4}(X)+h_{5}(X)}
$$

$$ g_{1}(X) = |x_{1}^{2}+x_{2}^{2}+x_{3}^{2}+x_{4}^{2}+x_{5}^{2}-10|$$
$$ g_{2}(X) = |x_{2}x{3}-5x_{4}x_{5}| $$
$$ g_{3}(X) = |x_{1}^{3}+x_{2}^{3}+1| $$
$$
h_{i}(x)= \left\{ \begin{array}{lcc}
             -u_{i}-x_{i} &   si  & x_{i} \leq -u_{i} \\
             \\ 0 &  si & -u_{i} \leq x_{i} < u_{i} \\
             \\ x_{i}-u_{i} &  si  & x_{i} \geq u_{1}
             \end{array}
   \right.
$$

##### Considere, además, las siguientes tres soluciones al problema relajado:

<img src="images/sol_iniciales.png">

##### Implemente un algoritmo (en Python o Matlab) que realice combinaciones de las soluciones utilizando el operador de cruce uniforme de Algoritmos Genéticos.

A continuación se encuentra la implementación

In [2]:
### creacion de funciones del problema relajado

def g1(X):
    return abs(np.dot(X,X)-10)

def g2(X):
    return abs(X[1]*X[2]-5*X[3]*X[4])

def g3(X):
    return abs(X[0]**3+X[1]**3+1)

def h(i,X):
    
    u = [2.3, 2.3, 3.2, 3.2, 3.2]
    
    if X[i]<= -u[i]:
        
        res = -u[i]-X[i]
        
    elif ((-u[i]<=X[i]) and (X[i]<=u[i])):
        
        res = 0
        
    elif X[i]>=u[i]:
        
        res = X[i]-u[i]
    
    return res

def FO(X):
    return math.e**(X[0]*X[1]*X[2]*X[3]*X[4])+math.e**(g1(X)+g2(X)+g3(X)+h(0,X)+h(1,X)+h(2,X)+h(3,X)+h(4,X))

In [3]:
### Soluciones iniciales dadas en el enunciado del examen
S1 = [-1.48, -0.07, -2.36, 1.13, 1.24]
S2 = [-0.49, -1.78, 2.01, 0.58, 0.38]
S3 = [0.27, 2.20, -0.32, 0.33, 2.47]

### se crea una unica lista con todas las soluciones
soluciones = [S1, S2, S3]

In [4]:
### Evaluamos la funcion objetivo de las soluciones
print(f'Solucion {S1} - FO: {round(FO(S1),2)}')
print(f'Solucion {S2} - FO: {round(FO(S2),2)}')
print(f'Solucion {S3} - FO: {round(FO(S3),2)}')

Solucion [-1.48, -0.07, -2.36, 1.13, 1.24] - FO: 15715.14
Solucion [-0.49, -1.78, 2.01, 0.58, 0.38] - FO: 99490.34
Solucion [0.27, 2.2, -0.32, 0.33, 2.47] - FO: 47312402.01


#### Cruce uniforme

En la siguiente celda se realiza el cruce uniforma entre dos individuos de la población (en este caso, la población corresponde a las soluciones iniciales dadas en el enunciado del parcial). Así, para cada par de soluciones, se itera en cada uno de sus genes y se selecciona de manera aleatoria uniforme un gen de alguno de los dos padres, por lo tanto realizar varias veces el cruce permite obtener diferentes hijos, por lo tanto, para cada par de soluciones, se realiza k veces el cruce uniforme, y finalmente se almacena la mejor solución encontrada

In [5]:
### Esta funcion corresponde a la implementacion del algoritmo que realiza las combinaciones de las soluciones
### realizando cruce uniforme
def cruce_uniforme(sol1, sol2):
    hijo1 = []
    hijo2 = []
    
    n = len(sol1)
    
    ### para los dos posibles hijos se le asignan los
    ### genes respectivamente de una forma uniforme
    ### es decir, se genera de forma aleatoria uniforme
    ### el gen de uno de los dos padres
    for i in range(n):
        if choice([1,2])==1:
            hijo1.append(sol1[i])
            hijo2.append(sol2[i])
        else:
            hijo1.append(sol2[i])
            hijo2.append(sol1[i])

    FO_h1 = FO(hijo1)
    FO_h2 = FO(hijo2)
    
    ### de los dos hijos, se obtiene el mejor, y este es
    ### el resultado del cruce
    best_hijo = None
    best_FO = np.inf
    if FO_h1 < FO_h2:
        best_hijo = hijo1
        best_FO = FO_h1
    else: 
        best_hijo = hijo2
        best_FO = FO_h2
    
    return best_hijo, best_FO

In [6]:
### El siguiente codigo corresponde al uso la funcion implementada para realizar cruces uniformes entre pares de soluciones

best_solucion = None
best_FO = np.inf

for s in soluciones:
    if FO(s)< best_FO:
        best_FO = FO(s)
        best_solucion = s
        
### se van a realizar k cruces uniformes para cada par de padres
N = len(soluciones)
k = 20
for i in range(N):
    for j in range(i,N):
        if not i==j:
            for cr in range(k):
                best_hijo_c, best_FO_c = cruce_uniforme(soluciones[i], soluciones[j])
                print(f'cruce {soluciones[i]}-{soluciones[j]} - hijo: {best_hijo_c} - FO {round(best_FO_c,2)}')
                if best_FO_c < best_FO:
                    best_FO = best_FO_c
                    besto_solucion = best_hijo_c
                    print('***'*20)
                    print('Se obtuvo una mejor solucion')
                    print('***'*20)

print('***'*20)
print(f'La mejor FO fue {round(best_FO,2)} con la solucion {besto_solucion}')

cruce [-1.48, -0.07, -2.36, 1.13, 1.24]-[-0.49, -1.78, 2.01, 0.58, 0.38] - hijo: [-0.49, -0.07, -2.36, 1.13, 1.24] - FO 8900.02
************************************************************
Se obtuvo una mejor solucion
************************************************************
cruce [-1.48, -0.07, -2.36, 1.13, 1.24]-[-0.49, -1.78, 2.01, 0.58, 0.38] - hijo: [-1.48, -0.07, 2.01, 0.58, 0.38] - FO 871.17
************************************************************
Se obtuvo una mejor solucion
************************************************************
cruce [-1.48, -0.07, -2.36, 1.13, 1.24]-[-0.49, -1.78, 2.01, 0.58, 0.38] - hijo: [-1.48, -0.07, -2.36, 0.58, 1.24] - FO 418.23
************************************************************
Se obtuvo una mejor solucion
************************************************************
cruce [-1.48, -0.07, -2.36, 1.13, 1.24]-[-0.49, -1.78, 2.01, 0.58, 0.38] - hijo: [-0.49, -0.07, -2.36, 0.58, 0.38] - FO 251.47
***************************************

#### Cruce por un punto

Adicional al cruce uniforme, se realiza la programación de cruce por un punto, por lo que para cada par de soluciones, se consideran todas las posibilidades de cruce de un punto, y finalmente se guarda la mejor solución obtenida

In [7]:
### Funcion para realizar el cruce en un punto
def cruce_punto(sol1, sol2, punto):
    
    ### se parten ambas soluciones en el punto dado y se
    ### crean los dos posibles hijos
    hijo1 = [*sol1[:punto],*sol2[punto:]]
    hijo2 = [*sol2[:punto],*sol1[punto:]]
    
    FO_h1 = FO(hijo1)
    FO_h2 = FO(hijo2)
    
    
    ### de los dos hijos, se obtiene el mejor, y este es
    ### el resultado del cruce    
    best_hijo = None
    best_FO = np.inf
    if FO_h1 < FO_h2:
        best_hijo = hijo1
        best_FO = FO_h1
    else: 
        best_hijo = hijo2
        best_FO = FO_h2
        
    return best_hijo, best_FO

In [8]:
### Combonaciones (cruce) de soluciones utilizando cruce en un punto
best_solucion = None
best_FO = np.inf

for s in soluciones:
    if FO(s)< best_FO:
        best_FO = FO(s)
        best_solucion = s

n_size = 5 ### tamano del individuo
N = len(soluciones) ### Cantidad de soluciones

### Para cada punto de cruce, realizar todas las combinaciones posibles
puntos_Cruce = [1,2,3]

for p in puntos_Cruce:
    for i in range(N):
        for j in range(i,N):
            if not i==j:
                best_hijo_c, best_FO_c = cruce_punto(soluciones[i], soluciones[j], p)
                print(f'Cruce {soluciones[i]} y {soluciones[j]} en punto {p} - hijo: {best_hijo_c} - FO: {round(best_FO_c,2)} ')
                if best_FO_c < best_FO:
                    best_FO = best_FO_c
                    besto_solucion = best_hijo_c
                    print('***'*20)
                    print('Se obtuvo una mejor solucion')
                    print('***'*20)
                    
print('***'*20)
print(f'La mejor FO fue {round(best_FO,2)} con la solucion {besto_solucion}')

Cruce [-1.48, -0.07, -2.36, 1.13, 1.24] y [-0.49, -1.78, 2.01, 0.58, 0.38] en punto 1 - hijo: [-0.49, -0.07, -2.36, 1.13, 1.24] - FO: 8900.02 
************************************************************
Se obtuvo una mejor solucion
************************************************************
Cruce [-1.48, -0.07, -2.36, 1.13, 1.24] y [0.27, 2.2, -0.32, 0.33, 2.47] en punto 1 - hijo: [0.27, -0.07, -2.36, 1.13, 1.24] - FO: 12068.19 
Cruce [-0.49, -1.78, 2.01, 0.58, 0.38] y [0.27, 2.2, -0.32, 0.33, 2.47] en punto 1 - hijo: [0.27, -1.78, 2.01, 0.58, 0.38] - FO: 102506.03 
Cruce [-1.48, -0.07, -2.36, 1.13, 1.24] y [-0.49, -1.78, 2.01, 0.58, 0.38] en punto 2 - hijo: [-1.48, -0.07, 2.01, 0.58, 0.38] - FO: 871.17 
************************************************************
Se obtuvo una mejor solucion
************************************************************
Cruce [-1.48, -0.07, -2.36, 1.13, 1.24] y [0.27, 2.2, -0.32, 0.33, 2.47] en punto 2 - hijo: [-1.48, -0.07, -0.32, 0.33, 2.47] - FO: 2